In [53]:
from datasets import load_dataset
import pandas as pd
dolly = load_dataset("databricks/databricks-dolly-15k")["train"]
dolly = pd.DataFrame(dolly)
alpaca = pd.read_json("../stanford_alpaca/alpaca_data.json")


In [54]:
dolly.drop(columns=["category"], inplace=True)
alpaca.columns = ["instruction", "context", "response"]

In [55]:
proportions = {"alpaca": 0.7, "dolly": 0.3, "train": 0.8, "validation": 0.2}
total_size = len(alpaca) + len(dolly)
total_size

67013

In [58]:
train_size = int(proportions["train"] * total_size)
validation_size = int(proportions["validation"] * total_size)
train_size + validation_size == total_size - 1

True

In [59]:
import numpy as np
alpaca_train_indices = np.random.randint(0, len(alpaca), size=int(train_size * proportions["alpaca"]))
dolly_train_indices = np.random.randint(0, len(dolly), size=int(train_size * proportions["dolly"]))                          
extract_alpaca_train = alpaca.iloc[alpaca_train_indices]
extract_dolly_train = dolly.iloc[dolly_train_indices]

alpaca_remaining = alpaca.drop(index=alpaca_train_indices)
dolly_remaining = dolly.drop(index=dolly_train_indices)

In [60]:
extract_alpaca_train

,instruction,context,response
47914,Create a conversation about July 4th.,,"A: Hey, it's July 4th! What fun things did you..."
36473,Use the input to describe a historic event.,The Battle of Verdun,The Battle of Verdun was one of the deadliest ...
46499,Describe a recurring dream you often have.,,I often have a dream where I'm running through...
46486,Come up with a mnemonic for remembering the da...,No input,A mnemonic for remembering the days of the wee...
35792,Describe the significance of this event in his...,The discovery of penicillin,The discovery of penicillin is considered one ...
...,...,...,...
19213,Generate a headline for an article discussing ...,,The Correlation Between Plant-Based Diet and R...
51104,Create an online advertisement that promotes a...,Seafood caught in a dolphin-friendly net,"Seafood lovers, it's time to make a difference..."
38148,Generate an antonym for 'tranquil',,Agitated.
45623,List 3 methods for solving a Rubik's cube,,Three methods of solving a Rubik's cube are th...


In [61]:
alpaca_validation_indices = np.random.randint(0, len(alpaca_remaining), size=int(validation_size * proportions["alpaca"]))
dolly_validation_indices = np.random.randint(0, len(dolly_remaining), size=int(validation_size * proportions["dolly"]))

try:
    extract_alpaca_validation = alpaca.iloc[alpaca_validation_indices]
    extract_dolly_validation = dolly.iloc[dolly_validation_indices]
except IndexError as e:
    print("Pas assez long")

In [62]:
extract_alpaca_validation.reset_index(inplace=True)
extract_alpaca_train.reset_index(inplace=True)
extract_dolly_train.reset_index(inplace=True)
extract_dolly_validation.reset_index(inplace=True)

In [64]:
extract_alpaca_train.drop(columns=["index"], inplace=True)
extract_alpaca_validation.drop(columns=["index"], inplace=True)
extract_dolly_train.drop(columns=["index"], inplace=True)
extract_dolly_validation.drop(columns=["index"], inplace=True)

/tmp/ipykernel_795698/963235013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_alpaca_train.drop(columns=["index"], inplace=True)
/tmp/ipykernel_795698/963235013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_alpaca_validation.drop(columns=["index"], inplace=True)
/tmp/ipykernel_795698/963235013.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_dolly_train.drop(columns=["index"], inplace=True)
/tmp/i

In [66]:
extract_alpaca_train

,instruction,context,response
0,Create a conversation about July 4th.,,"A: Hey, it's July 4th! What fun things did you..."
1,Use the input to describe a historic event.,The Battle of Verdun,The Battle of Verdun was one of the deadliest ...
2,Describe a recurring dream you often have.,,I often have a dream where I'm running through...
3,Come up with a mnemonic for remembering the da...,No input,A mnemonic for remembering the days of the wee...
4,Describe the significance of this event in his...,The discovery of penicillin,The discovery of penicillin is considered one ...
...,...,...,...
37522,Generate a headline for an article discussing ...,,The Correlation Between Plant-Based Diet and R...
37523,Create an online advertisement that promotes a...,Seafood caught in a dolphin-friendly net,"Seafood lovers, it's time to make a difference..."
37524,Generate an antonym for 'tranquil',,Agitated.
37525,List 3 methods for solving a Rubik's cube,,Three methods of solving a Rubik's cube are th...


In [70]:
train = pd.concat([extract_dolly_train, extract_alpaca_train], axis=0)
train = train.sample(frac=1, random_state=42)
train.to_json("./train_mixed.json", orient="records")

In [71]:
validation = pd.concat([extract_dolly_validation, extract_alpaca_validation], axis=0)
validation = validation.sample(frac=1, random_state=42)
validation.to_json("./validation_mixed.json", orient="records")